In [1]:
import warnings
import pandas as pd
import numpy as np
import scipy.stats as st
from statsmodels.formula.api import ols
warnings.filterwarnings('ignore')

## Set6

In [2]:
df6 = pd.read_csv(path+'DataSet_06.csv')
df6.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,sqft_living15,sqft_lot15
0,2597000006,20150309T000000,347500.0,3,1.50,1180,8353,1.0,0,0,3,7,1180,0,1960,0,98155,1710,8748
1,8813400345,20150414T000000,575000.0,2,1.00,980,3663,1.0,0,0,5,7,980,0,1909,0,98105,1620,3706
2,226059106,20150102T000000,489500.0,3,1.75,2090,65558,1.0,0,0,3,8,1330,760,1977,0,98072,2450,47178
3,8645500270,20141028T000000,246000.0,4,1.75,1720,7455,1.0,0,0,4,7,1020,700,1963,0,98058,1720,7700
4,7010700550,20141114T000000,595000.0,3,2.50,2030,5100,2.0,0,0,3,7,2030,0,2008,0,98199,1790,4380


In [8]:
# =============================================================================
# 1. O
# =============================================================================

q1 = df6[['price', 'waterfront']]

wf_y = q1[q1['waterfront']==1]['price'].mean()
wf_n = q1[q1['waterfront']==0]['price'].mean()
res = wf_y-wf_n
print(f'{np.floor(res):.0f}')

1167272


In [27]:
# =============================================================================
# 2. O
# =============================================================================
q2 = df6[['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'yr_built']]
mx = abs(q2.corr()[['price']].drop('price')).idxmax()
mn = abs(q2.corr()[['price']].drop('price')).idxmin()
print(f'{mx[0]}, {mn[0]}')

sqft_living, yr_built


In [37]:
q3.columns

Index(['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'sqft_living15',
       'sqft_lot15'],
      dtype='object')

In [56]:
# =============================================================================
# 3. ☆
# =============================================================================

q3 = df6.drop(['id', 'date', 'zipcode'], axis=1).copy()
# y = q3[['price']]
# X = q3.drop(['price'], axis=1)

from statsmodels.formula.api import ols

var = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'sqft_living15',
       'sqft_lot15']

form = 'price~' + '+'.join(var)
ols = ols(form, q3).fit()
res1 = ols.params[ols.pvalues < 0.05].drop('Intercept')
res2 = len(res1[res1 < 0])

print(f'{len(res1)}, {res2}')

13, 2


## Set7

In [57]:
df7 = pd.read_csv(path+'DataSet_07.csv')
df7.head()

,Serial_No,GRE,TOEFL,University_Rating,SOP,LOR,CGPA,Research,Chance_of_Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


In [66]:
# =============================================================================
# 1. O
# =============================================================================
q1 = df7[['GRE', 'TOEFL', 'CGPA','Chance_of_Admit']]
res1 = q1.corr()[['Chance_of_Admit']].drop('Chance_of_Admit').max()[0]
print(round(res1, 3))

0.873


In [86]:
# =============================================================================
# 2. O
# =============================================================================
q2 = df7[['GRE', 'CGPA']]
mean = q2['GRE'].mean()

# 미만 : 0, 이상 : 1
q2['GRE'] = np.where(q2[['GRE']]<mean, 0, 1)

from scipy.stats import ttest_ind

out = ttest_ind(q2[q2['GRE']==1]['CGPA'], q2[q2['GRE']==0]['CGPA'])
print(round(out[0], 2))

19.44


In [109]:
# =============================================================================
# 3. △
# =============================================================================
seed = 123
q3 = df7.drop(['Serial_No'], axis=1)

# 합격 : 1, 불합격:0
q3['Chance_of_Admit'] = np.where(q3['Chance_of_Admit']>0.5, 1, 0)
y = q3[['Chance_of_Admit']]
X = q3.drop(['Chance_of_Admit'], axis=1)

from sklearn.linear_model import LogisticRegression

# help(LogisticRegression)
lr = LogisticRegression(solver='liblinear', random_state=seed).fit(X, y)
res1 = pd.Series(abs(lr.coef_[0]), index=X.columns).idxmax()
res2 = pd.Series(abs(lr.coef_[0]), index=X.columns).max()
print(f'{res1}, {round(res2, 2)}')

CGPA, 1.98


## Set8

In [110]:
df8 = pd.read_csv(path+'DataSet_08.csv')
df8.head()

,RandD_Spend,Administration,Marketing_Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [170]:
# =============================================================================
# 1. O
# =============================================================================
q1 = df8['State']

q1 = pd.DataFrame(q1.value_counts().sort_index())
q1['State'] = round(q1['State']/q1['State'].sum(), 2)
print(q1.values.reshape(-1))

[0.34 0.32 0.34]


In [182]:
# =============================================================================
# 2. O
# =============================================================================
q2 = df8.groupby('State')['Profit'].mean()
res2 = q2.max()-q2.min()
print(f'{np.floor(res):.0f}')

1167272


In [200]:
# =============================================================================
# 3. 
# =============================================================================
var = ['RandD_Spend', 'Administration', 'Marketing_Spend']
st_lst = df8['State'].unique()


from statsmodels.formula.api import ols

lst = []
for st in st_lst:
    q3 = df8[df8['State']==st]
    form = 'Profit~'+ '+'.join(var)

    reg = ols(form, df8).fit()
    y_hat = reg.predict(q3[var])
    mape = sum(abs(q3['Profit']-y_hat)/q3['Profit']) * 100 / len(q3)
    lst.append([st, round(mape, 2)])
    
print(pd.DataFrame(lst, columns=['State','MAPE']).min()

State    California
MAPE           5.68
dtype: object


In [206]:
state_list=df8.State.unique()

from sklearn.linear_model import LinearRegression

q3_out=[]
for i in state_list:
    temp=df8[df8.State == i]
    lm=LinearRegression().fit(temp[var], temp['Profit'])
    pred=lm.predict(temp[var])
    # Σ ( | y - y ̂ | / y ) * 100/n
    mape=(abs(temp['Profit'] - pred)/temp['Profit']).sum() * 100 / len(temp)
    q3_out.append([i, mape])
    
# 2.주별로 계산된 학습오차 중 MAPE 기준으로 가장 낮은 오차를 보이는 주는 어느
# 주이고 그 값은 무엇인가?

q3_out=pd.DataFrame(q3_out, columns=['state', 'mape'] )
q3_out.loc[q3_out.mape.idxmin(),:]

state     Florida
mape     5.706713
Name: 2, dtype: object

## Set9

In [207]:
df9 = pd.read_csv(path+'DataSet_09.csv')
df9.head()

,satisfaction,Gender,Age,Customer_Type,Class,Flight_Distance,Seat_comfort,Food_and_drink,Inflight_wifi_service,Inflight_entertainment,Onboard_service,Leg_room_service,Baggage_handling,Cleanliness,Departure_Delay_in_Minutes,Arrival_Delay_in_Minutes
0,satisfied,Female,19.0,Loyal Customer,Eco,1981.0,4.0,4.0,1.0,4.0,4.0,4.0,4.0,4.0,3.0,28.0
1,dissatisfied,Female,64.0,Loyal Customer,Eco,2847.0,1.0,1.0,1.0,2.0,5.0,3.0,4.0,4.0,111.0,133.0
2,dissatisfied,Male,55.0,Loyal Customer,Eco,3790.0,1.0,1.0,2.0,1.0,4.0,4.0,4.0,4.0,2.0,15.0
3,satisfied,Female,39.0,Loyal Customer,Eco,1741.0,4.0,4.0,5.0,4.0,1.0,1.0,5.0,5.0,2.0,0.0
4,dissatisfied,Female,29.0,Loyal Customer,Eco,3829.0,2.0,1.0,1.0,4.0,2.0,4.0,3.0,1.0,365.0,383.0


In [211]:
# =============================================================================
# 1. 
# =============================================================================
df9.isna().sum().sum()

5

In [229]:
# =============================================================================
# 2. X, chi2를 수행할 때는 crosstab 만들 것!!
# =============================================================================
from scipy.stats import chi2_contingency

q2 = df9.dropna()
q2['Age_gr']=np.where(q2.Age <= 20, 10,
                np.where(q2.Age <= 30, 20,
                   np.where(q2.Age <= 40, 30,
                     np.where(q2.Age <= 50, 40,
                       np.where(q2.Age <= 60, 50, 60)))))

var_lst = ['Age_gr', 'Gender', 'Customer_Type', 'Class']
# q2['Gender'] = q2['Gender'].replace('Male', 0).replace('Female', 1)

for var in var_lst:
    tab = pd.crosstab(index=q2[var], columns=q2['satisfaction'])
    out = chi2_contingency(tab)
    if out[1] < 0.05:
        print(f'{var}, {np.floor(out[0]):.0f}')

Gender, 1066


In [236]:
# =============================================================================
# 3. O
# =============================================================================
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

seed = 123
df9 = df9.dropna()
X = df9[['Flight_Distance', 'Seat_comfort', 'Food_and_drink', 'Inflight_wifi_service', 
'Inflight_entertainment','Onboard_service', 'Leg_room_service', 'Baggage_handling',
'Cleanliness', 'Departure_Delay_in_Minutes', 'Arrival_Delay_in_Minutes']]
y = df9[['satisfaction']]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state = seed, test_size = 0.3
)

lr = LogisticRegression().fit(X_train, y_train)

from sklearn.metrics import f1_score
pred = lr.predict(X_test)
res3 = f1_score(y_test, pred, pos_label='dissatisfied')
print(round(res3, 3))

0.749


## Set10

In [2]:
df10 = pd.read_csv(path+'DataSet_10.csv')
df10 = df10.dropna(axis=1)
df10.head()

,model,engine_power,age_in_days,km,previous_owners,price
0,lounge,51,882,25000,1,8900
1,pop,51,1186,32500,1,8800
2,sport,74,4658,142228,1,4200
3,lounge,51,2739,160000,1,6000
4,pop,73,3074,106880,1,5700


In [39]:
# =============================================================================
# 1. O
# =============================================================================
q1 = df10[(df10['previous_owners']==1)&(df10['engine_power']==51)]

q1['km/day'] = q1['km']/q1['age_in_days']
tab = q1.groupby('model')[['km/day']].mean()
round(tab.min()/tab.max(), 2)

km/day    0.98
dtype: float64

In [47]:
# =============================================================================
# 2. O
# =============================================================================
# lounge, pop
q2 = df10.copy()
q2['km/day'] = q2['km']/q2['age_in_days']

from scipy.stats import ttest_ind

lounge = q2[q2['model']==tab['km/day'].idxmax()][['km/day']].dropna()
pop = q2[q2['model']==tab['km/day'].idxmin()][['km/day']].dropna()

out = ttest_ind(lounge, pop, equal_var=True)

if out[1] < 0.05:
    res = 'Y'
else:
    res = 'N'
print(f'{np.floor(out[1]*100)/100}, {res}')

[0.48], N


In [81]:
# =============================================================================
# 3. X
# =============================================================================
X = df10[['engine_power', 'age_in_days', 'km']]
y = df10[['price']]

from sklearn.linear_model import LinearRegression

lr = LinearRegression().fit(X, y)
res3 = lr.predict([[51, 400, 9500]])
print(f'{np.floor(res3)[0][0]:.0f}')

10469
